In [17]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

import plotly.io as pio
pio.renderers.default = "plotly_mimetype"

from visualize_relations import *
from micro_segmentation import *

## Testdata Preparation
 Data preprocessing in``data/preprocess.sh``:

Use SWaT trace SWaT.A6_Dec 2019 ``Dec2019_00000_20191206100500.pcap``, filter traffic of PLCs and SCADA.


In [ ]:
# SWaT pcap preprocessed
pcap_file = "../data/dump.pcap"
# generated using parse_dump in visualize_relations.py
csv_flow_file = "../data/flows.csv"

## VPN-enhanced Micro-Segmentation

### (1) Visualize and Identify communication relations

the logic behind the identification & visualization of communication relations can be found in ``flow_analysis.py`` and``visualize_relations.py``.

``parse_dump`` parses a network dump file and extracts its communication flows using [NFStream](https://www.nfstream.org) (stores flows as csv as well)

In [19]:
df_flows = pd.read_csv(csv_flow_file, sep=',')
df_flows.head(10)

,id,expiration_id,src_ip,src_mac,src_oui,src_port,dst_ip,dst_mac,dst_oui,dst_port,protocol,ip_version,vlan_id,tunnel_id,bidirectional_first_seen_ms,bidirectional_last_seen_ms,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_first_seen_ms,src2dst_last_seen_ms,src2dst_duration_ms,src2dst_packets,src2dst_bytes,dst2src_first_seen_ms,dst2src_last_seen_ms,dst2src_duration_ms,dst2src_packets,dst2src_bytes,application_name,application_category_name,application_is_guessed,application_confidence,requested_server_name,client_fingerprint,server_fingerprint,user_agent,content_type
0,0,0,192.168.1.30,00:1d:9c:c8:bd:f2,00:1d:9c,44818,192.168.1.100,00:1d:9c:c6:72:e8,00:1d:9c,49401,6,4,0,0,1575597904374,1575597907113,2739,39,5047,1575597904374,1575597907113,2739,23,3493,1575597904411,1575597907106,2695,16,1554,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN
1,1,0,192.168.1.30,00:1d:9c:c8:bd:f2,00:1d:9c,60704,192.168.1.20,00:1d:9c:c8:bc:46,00:1d:9c,44818,6,4,0,0,1575597904374,1575597907149,2775,1294,117824,1575597904374,1575597907149,2775,647,60252,1575597904374,1575597907149,2775,647,57572,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN
2,2,0,192.168.1.40,00:1d:9c:c7:fa:2c,00:1d:9c,44818,192.168.1.50,00:1d:9c:c8:f4:b9,00:1d:9c,63084,6,4,1,0,1575597904377,1575597907147,2770,804,77676,1575597904377,1575597907147,2770,402,39316,1575597904377,1575597907147,2770,402,38360,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN
3,3,0,192.168.1.50,00:1d:9c:c8:f4:b9,00:1d:9c,44818,192.168.1.20,00:1d:9c:c8:bc:46,00:1d:9c,60936,6,4,0,0,1575597904382,1575597907149,2767,1044,99196,1575597904382,1575597907149,2767,522,53224,1575597904382,1575597907149,2767,522,45972,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN
4,4,0,192.168.1.50,00:1d:9c:c8:f4:b9,00:1d:9c,44818,192.168.1.100,00:1d:9c:c6:72:e8,00:1d:9c,49875,6,4,1,0,1575597904382,1575597907128,2746,24,4316,1575597904382,1575597907128,2746,13,3324,1575597904412,1575597907106,2694,11,992,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN
5,5,0,192.168.1.60,00:1d:9c:c7:fa:2d,00:1d:9c,59072,192.168.1.10,00:1d:9c:c8:bd:e7,00:1d:9c,44818,6,4,1,0,1575597904383,1575597907139,2756,440,43576,1575597904383,1575597907139,2756,220,20276,1575597904384,1575597907139,2755,220,23300,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN
6,6,0,192.168.1.40,00:1d:9c:c7:fa:2c,00:1d:9c,44818,192.168.1.20,00:1d:9c:c8:bc:46,00:1d:9c,60892,6,4,0,0,1575597904387,1575597907147,2760,338,32140,1575597904387,1575597907147,2760,169,17260,1575597904387,1575597907147,2760,169,14880,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN
7,7,0,192.168.1.50,00:1d:9c:c8:f4:b9,00:1d:9c,44818,192.168.1.40,00:1d:9c:c7:fa:2c,00:1d:9c,59170,6,4,0,0,1575597904392,1575597907146,2754,1018,89628,1575597904392,1575597907146,2754,509,44772,1575597904392,1575597907146,2754,509,44856,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN
8,8,0,192.168.1.20,00:1d:9c:c8:bc:46,00:1d:9c,44818,192.168.1.100,00:1d:9c:c6:72:e8,00:1d:9c,49400,6,4,1,0,1575597904376,1575597907123,2747,36,4917,1575597904376,1575597907123,2747,21,3411,1575597904412,1575597907106,2694,15,1506,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN
9,9,0,192.168.1.40,00:1d:9c:c7:fa:2c,00:1d:9c,44818,192.168.1.50,00:1d:9c:c8:f4:b9,00:1d:9c,63084,6,4,0,0,1575597904377,1575597907147,2770,804,74460,1575597904377,1575597907147,2770,402,37708,1575597904377,1575597907147,2770,402,36752,EthernetIP,Network,0,6,NaN,NaN,NaN,NaN,NaN


In [21]:
df_selected = df_flows[["src_ip", "dst_ip", "src_port", "dst_port", "protocol", "src2dst_packets", "dst2src_packets", "application_name"]]
df_grouped = df_selected.groupby(["src_ip", "dst_ip", "src_port", "dst_port", "protocol", "application_name"]).agg({
    "src2dst_packets": "sum",   # Summing packets from src to dst
    "dst2src_packets": "sum"    # Summing packets from dst to src
}).reset_index()
df_grouped["bi-directional"] = df_grouped["dst2src_packets"] > 0

# manual positions to visualize SWaT testcase
pos = {
'192.168.1.100': [825, 1000],
'192.168.1.10': [200, 600],
'192.168.1.20': [450, 600],
'192.168.1.30': [700, 600],
'192.168.1.40': [950, 600],
'192.168.1.50': [1200, 600],
'192.168.1.60': [1450, 600]
}
visualize_flows_df(df_grouped, pos=pos)

### (2) Micro-segmentation

Micro-segments can vary in granularity, resulting in different sets of rules.
For example, for device-level segmentation, we block or allow communication at the device level, making decisions based on source and destination IP.
For application-level segmentation, we further consider protocol and ports, i.e., the specific application, making decisions more granular.

In [22]:
PLC6 = "192.168.1.60"
df_PLC6_host_level = host_level_segmentation(PLC6, df_flows)
df_PLC6_app_level = application_level_segmentation(PLC6, df_flows)


Unique Host-level Communication Flows for 192.168.1.60:
           ip
 192.168.1.10
 192.168.1.20
 192.168.1.30
192.168.1.100


Unique Application-level Communication Flows for 192.168.1.60:
       src_ip       dst_ip  src_port  dst_port  protocol application_name  bi-directional
 192.168.1.10 192.168.1.60      <NA>     44818         6       EthernetIP            True
 192.168.1.20 192.168.1.60     44818      <NA>         6       EthernetIP            True
 192.168.1.30 192.168.1.60      2222      2222        17       EthernetIP           False
 192.168.1.30 192.168.1.60      <NA>     44818         6       EthernetIP            True
 192.168.1.60 192.168.1.10      2222      2222        17       EthernetIP            True
 192.168.1.60 192.168.1.10      <NA>     44818         6       EthernetIP            True
 192.168.1.60 192.168.1.20      2222      2222        17       EthernetIP           False
192.168.1.100 192.168.1.60      <NA>     44818         6       EthernetIP            True

### (2.1) Convert to firewall rules (iptables)

Example:

```
Unique Host-level Communication Flows for 192.168.1.60:
           ip
 192.168.1.10
 192.168.1.20
 192.168.1.30
192.168.1.100


iptables -A INPUT -s 192.168.1.10 -j ACCEPT
iptables -A OUTPUT -d 192.168.1.10 -j ACCEPT
iptables -A INPUT -s 192.168.1.20 -j ACCEPT
iptables -A OUTPUT -d 192.168.1.20 -j ACCEPT
iptables -A INPUT -s 192.168.1.30 -j ACCEPT
iptables -A OUTPUT -d 192.168.1.30 -j ACCEPT
iptables -A INPUT -s 192.168.1.100 -j ACCEPT
iptables -A OUTPUT -d 192.168.1.100 -j ACCEPT



```



```
Unique Application-level Communication Flows for 192.168.1.60:
       src_ip       dst_ip  src_port  dst_port  protocol application_name  bi-directional
 192.168.1.10 192.168.1.60      <NA>     44818         6       EthernetIP            True
 192.168.1.20 192.168.1.60     44818      <NA>         6       EthernetIP            True
 192.168.1.30 192.168.1.60      2222      2222        17       EthernetIP           False
 192.168.1.30 192.168.1.60      <NA>     44818         6       EthernetIP            True
 192.168.1.60 192.168.1.10      2222      2222        17       EthernetIP            True
 192.168.1.60 192.168.1.10      <NA>     44818         6       EthernetIP            True
 192.168.1.60 192.168.1.20      2222      2222        17       EthernetIP           False
192.168.1.100 192.168.1.60      <NA>     44818         6       EthernetIP            True


iptables -A INPUT -p tcp -s 192.168.1.10 -d 192.168.1.60 --sport any --dport 44818 -j ACCEPT
iptables -A OUTPUT -p tcp -s 192.168.1.60 -d 192.168.1.10 --sport 44818 --dport any -j ACCEPT
iptables -A INPUT -p tcp -s 192.168.1.20 -d 192.168.1.60 --sport 44818 --dport any -j ACCEPT
iptables -A OUTPUT -p tcp -s 192.168.1.60 -d 192.168.1.20 --sport any --dport 44818 -j ACCEPT
iptables -A INPUT -p udp -s 192.168.1.30 -d 192.168.1.60 --sport 2222 --dport 2222 -j ACCEPT
iptables -A INPUT -p tcp -s 192.168.1.30 -d 192.168.1.60 --sport any --dport 44818 -j ACCEPT
iptables -A OUTPUT -p tcp -s 192.168.1.60 -d 192.168.1.30 --sport 44818 --dport any -j ACCEPT
iptables -A INPUT -p udp -s 192.168.1.60 -d 192.168.1.10 --sport 2222 --dport 2222 -j ACCEPT
iptables -A OUTPUT -p udp -s 192.168.1.10 -d 192.168.1.60 --sport 2222 --dport 2222 -j ACCEPT
iptables -A INPUT -p tcp -s 192.168.1.60 -d 192.168.1.10 --sport any --dport 44818 -j ACCEPT
iptables -A OUTPUT -p tcp -s 192.168.1.10 -d 192.168.1.60 --sport 44818 --dport any -j ACCEPT
iptables -A OUTPUT -p udp -s 192.168.1.60 -d 192.168.1.20 --sport 2222 --dport 2222 -j ACCEPT
iptables -A INPUT -p tcp -s 192.168.1.100 -d 192.168.1.60 --sport any --dport 44818 -j ACCEPT
iptables -A OUTPUT -p tcp -s 192.168.1.60 -d 192.168.1.100 --sport 44818 --dport any -j ACCEPT
```




### (3) Selective VPN Encryption

Generate WireGuard configs (for different communication flows).


Example:
```

[Interface]
PrivateKey = SOAvevnjOpnrmF61BW1JXCNmJneTrsCZgKaSZw9afmU=
ListenPort = 51005
Address = 192.168.1.60/24

[Peer]
PublicKey = 8DsgqaEF/B3kgiJXZNNMNE2SwoupJZ8RsxgEp3546T0=
AllowedIPs = 192.168.1.10
```